In [6]:
import numpy as np
import matplotlib.pylab as plt
import scipy.constants as spc
from scipy.integrate import quad
'''
Need to move this to a .py script later on!
'''

'\nNeed to move this to a .py script later on!\n'

In [4]:
'''
Conventions 

v = wavenumber[cm^-1] --- number if waves per cm. v = f/c
tau = transmittance --- ratio of transmitted radiance to incident radiance
epsilon = emissivity --- ratio of emitted radiance at v to the radiance emitted by the black body at the same temperature
alpha = absorptivity --- fraction of incident radiance at v that is absorbed
g = line shape
f = voigt profile
'''

'\nConventions \n\nv = wavenumber[cm^-1] --- number if waves per cm. v = f/c\ntau = transmittance --- ratio of transmitted radiance to incident radiance\nepsilon = emissivity --- ratio of emitted radiance at v to the radiance emitted by the black body at the same temperature\nalpha = absorptivity --- fraction of incident radiance at v that is absorbed\n\n'

In [7]:
def wavenumber_center(pressure):
    P_0 = 1 #[atm] unsure
    pressureRatio = pressure/P_0
    #delta = 
    #v_0 =
    v_center = v_0 + delta*pressureRatio
    return v_center

def doppler_broadening(temperature):
    '''
    For pressures less than 0.1 atm, altitudes below 16 km
    '''
    T = temperature
    v_center = wavenumber_center(pressure) 
    #m = 
    sqrt = np.sqrt(2*spc.k*T/(m*spc.c**2))
    alpha_d = v_center*sqrt
    return alpha_d
    
def lorentz_broadening(pressure, temperature):
    P, T = pressure, temperature
    P_0 = 1 #[atm] unsure
    T_0 = 296 #[K]
    #alphaLorentz_a = 
    #alphaLorentz_s = 
    #q = 
    #gamma = 
    pressureRatio = P/P_0
    tempRatio = T_0/T
    alpha_lorentz = ((1-q)*alphaLorentz_a+q*alphaLorentz_s)*pressureRatio*tempRatio**gamma
    return alpha_lorentz

def integral_func(t, a, b):
    val = np.exp(-t**2)/((a - t)**2 + b**2)
    return val
    
def voigt_profile():
    alpha_doppler = doppler_broadening(pressure, temperature)
    alpha_lorentz = lorentz_broadening(temperature)
    #v = 
    v_center = wavenumber_center(pressure)  
    a = (v - v_center)/alpha_doppler
    b = alpha_lorentz/alpha_doppler
    integral = quad(integral_func, -np.inf, np.inf, )
    f = b*integral /(alpha_doppler*np.sqrt(np.pi)*np.pi)
    return f

def line_shape():
    #v =
    v_center = wavenumber_center(pressure)  
    f = voigt_profile()
    tanh = np.tanh(spc.h*spc.c*v/(2*spc.k*))
    tanh_center = np.tanh(spc.h*spc.c * v_center/(2*spc.k*))
    g = v * tanh * f/(v_center*tanh_center)
    return g

def line_intensity(temperature):
    T = temperature
    v_center = wavenumber_center(pressure)
    #S_0, T_0 = ,296
    #Q_v, Q_r = 
    #Q_v0, Q_r0 =
    #E_l = 
    exp_lower = np.exp(spc.h*spc.c*E_l/(spc.k*T))
    exp_lower0 = np.exp(spc.h*spc.c*E_l/(spc.k*T_0))
    exp_center = np.exp(spc.h*spc.c*v_center/(spc.k*T))
    exp_center0 = np.exp(spc.h*spc.c*v_center/(spc.k*T))
    S = S_0*Q_v0*Q_r0*exp_lower*exp_center/(Q_v*Q_r*exp_lower0*exp_center0)
    return S
    
def transmittance(pressure, temperature):
    P = pressure
    T = temperature
    #x = 
    #q = 
    g = line_shape()
    #S =
    numerator = -q*P*x*S*g
    denominator = spc.Boltzmann * T
    tau = np.exp(numerator/denominator)
    return tau

IndentationError: expected an indented block (<ipython-input-7-f2203b267e7d>, line 4)